In [73]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

## Importing Libraries and tokens

In [74]:
import os

import wandb
import torch

# load .env file
from dotenv import load_dotenv

from geo_model_tester import GeoModelTester
from image_data_handler_test import TestImageDataHandler
from best_run_loader import BestRunLoader
from wandb_downloader import WandbDownloader

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [75]:
WANDB_TOKEN = os.getenv("WANDB_TOKEN")
# Define where to run
env_path = "../../.env"
if not WANDB_TOKEN and os.path.exists(env_path):
    load_dotenv(env_path)
    WANDB_TOKEN = os.getenv("WANDB_TOKEN")

In [76]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")

    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


## Loading files from wandb

In [77]:
wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

True

In [78]:
# Setting for the right models to test
entity = "nlp_ls"
predict_coordinates = False
predict_regions = True
datasize = 79000  # 79000, 81505, 332786
data_augmentation = "base_augmentation"  # or "base_augmentation", "full_augmentation_v2"

# Automatic settings
project = "dspro2-predicting-region" if predict_regions else ("dspro2-predicting-coordinates" if predict_coordinates else "dspro2-predicting-country")
metric_name = "Best Validation Accuracy Top 1" if not predict_coordinates else "Best Validation Distance (km)"
metric_ascending = False if not predict_coordinates else True
file_names_to_download = [".pth", ".json"]
image_size = [180, 320] if datasize == 79000 else [80, 130]

downloader = WandbDownloader(entity, project, data_augmentation, datasize, image_size)
run_data = downloader.get_and_collect_best_runs(metric_name, file_names_to_download, metric_ascending=metric_ascending)

dspro2-predicting-region: Found 5 matching runs for datasize 79000 and base_augmentation.
Could not find test_data.pth for run uk9kolsi, looks like run b6tzuedd was deleted or is part of a different project.


In [79]:
# Print the validation accuracy for the top 1, 3, and 5 predictions
for j in range(1, min(len(run_data), 6)):
    print(f"\nBest Run {j}: {run_data[f'Best Run {j}']['id']}")
    if predict_coordinates:
        print(f"Best Validation Distance (km): ", run_data[f"Best Run {j}"]["metrics"]["Best Validation Distance (km)"])
    else:
        for i in [1, 3, 5]:
            try:
                print(f"Best Validation Accuracy Top {i}: ", run_data[f"Best Run {j}"]["metrics"][f"Best Validation Accuracy Top {i}"])
            except KeyError:
                print("No validation accuracy found.")


Best Run 1: l964tv8w
Best Validation Accuracy Top 1:  0.009746835443037977
Best Validation Accuracy Top 3:  0.02189873417721519
Best Validation Accuracy Top 5:  0.029746835443037977

Best Run 2: j3zik6vp
Best Validation Accuracy Top 1:  0.009683544303797469
Best Validation Accuracy Top 3:  0.01240506329113924
Best Validation Accuracy Top 5:  0.014746835443037974

Best Run 3: hd3f2sg1
Best Validation Accuracy Top 1:  0.009683544303797469
Best Validation Accuracy Top 3:  0.01240506329113924
Best Validation Accuracy Top 5:  0.014746835443037974

Best Run 4: jzr47aha
Best Validation Accuracy Top 1:  0.009556962025316456
Best Validation Accuracy Top 3:  0.010379746835443038
Best Validation Accuracy Top 5:  0.012974683544303795


In [80]:
run_data["Best Run 1"]["parameters"]

{'seed': 42,
 'epochs': 50,
 'optimizer': 'adamW',
 'batch_size': 200,
 'model_name': 'efficientnet_b1',
 'mapped_data': True,
 'dataset_size': 79000,
 'weight_decay': 0.01,
 'learning_rate': 0.001,
 'predict_regions': True,
 'input_image_size': [180, 320],
 'data_augmentation': 'base_augmentation',
 'different_regions': 4596,
 'dataset_identifier': '3d999c168bedf247468b758771d441ecfd202c97836bed006c225a4afbc8688a',
 'different_countries': 75,
 'predict_coordinates': False}

In [81]:
run_data["Best Run 1"]["files"]

{'wandb_manifest.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/artifact/926447535/wandb_manifest.json',
 'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/best_model_checkpointmodel_efficientnet_b1_lr_0.001_opt_adamW_weightDecay_0.01_imgSize_[180, 320]_predict_coordinates_False.pth',
 'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/run-20240625_181225-l964tv8w/country_to_index.json',
 'region_index_to_country_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/run-20240625_181225-l964tv8w/region_index_to_country_index.json',
 'region_index_to_middle_point.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/run-20240625_181225-l964tv8w/region_index_to_middle_point.json',
 'region_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/l964tv8w/run-20240625_181225-l964tv8w/region_to_index.json',
 'wandb-meta

## Loading data and creating data loader

In [82]:
cache = True

run = None

for i in range(min(len(run_data), 5)):
    run = run_data[f"Best Run {i+1}"]
    if run["files"].get("test_data", None) and run["files"].get("best_model", None):
        break
    else:
        run = None
        print(f"Run {i+1} does not contain the necessary files. Trying the next run...")

if run is None:
    raise Exception("No run with the necessary files found.")

# Creating Dataloaders with the classes
test_dataset = run["files"]["test_data"]
files = run["files"]
country_to_index = files.get("country_to_index.json", None)
region_to_index = files.get("region_to_index.json", None)
region_index_to_middle_point = files.get("region_index_to_middle_point.json", None)
region_index_to_country_index = files.get("region_index_to_country_index.json", None)

data_handler = TestImageDataHandler(test_dataset, country_to_index, region_to_index, region_index_to_middle_point, region_index_to_country_index, cache=cache)
test_dataloader = data_handler.test_loader
country_to_index = data_handler.country_to_index
region_to_index = data_handler.region_to_index
region_index_to_middle_point = data_handler.region_index_to_middle_point
region_index_to_country_index = data_handler.region_index_to_country_index

num_regions = data_handler.num_regions
num_countries = data_handler.num_countries

Loaded 75 countries.
Loaded 4596 regions.
Loaded 4596 region middle points.
Loaded 2676 region to country index mappings.
Loading test data from test_data.pth
Test data loaded from cache.


## Evaluating the model

In [83]:
num_classes = 3 if predict_coordinates else (num_regions if predict_regions else num_countries)

if num_classes == 0:
    raise ValueError("No classes detected. Please check the data.")

geo_model_tester = GeoModelTester(test_dataloader=test_dataloader, num_classes=num_classes, predict_coordinates=predict_coordinates, country_to_index=country_to_index, region_to_index=region_to_index, region_index_to_middle_point=region_index_to_middle_point, region_index_to_country_index=region_index_to_country_index, predict_regions=predict_regions if not predict_coordinates else None)

/Users/lukasstoeckli/GitLabProjects/dspro2/dspro2/5_evaluation/../4_modeling/geo_model_harness.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  self.region_middle_points = torch.tensor([coordinates_to_cartesian(*coordinate) for coordinate in self.region_middle_points], dtype=torch.float32).to(self.device)


In [84]:
# TODO: Test the model from best runs
# TODO: Show the different models with the best results (also do it for different data sizes and mapped/non-mapped data)
model_name = run["parameters"]["model_name"]
pretrained_weights = run["files"]["best_model"]

# Countries from 81k more mapped dataset, keep in sync with evaluating_models.ipynb
countries_only = ["Albania", "Argentina", "Australia", "Austria", "Bangladesh", "Belgium", "Bolivia, Plurinational State of", "Botswana", "Brazil", "Bulgaria", "Cambodia", "Canada", "Chile", "Colombia", "Croatia", "Czechia", "Denmark", "Dominican Republic", "Ecuador", "Estonia", "Eswatini", "Finland", "France", "Germany", "Ghana", "Greece", "Guatemala", "Hungary", "India", "Indonesia", "Ireland", "Israel", "Italy", "Japan", "Kenya", "Korea, Republic of", "Kyrgyzstan", "Lao People's Democratic Republic", "Latvia", "Lesotho", "Lithuania", "Malaysia", "Malta", "Mexico", "Montenegro", "Netherlands", "New Zealand", "Nigeria", "North Macedonia", "Norway", "Peru", "Philippines", "Poland", "Portugal", "Romania", "Russian Federation", "Rwanda", "Senegal", "Serbia", "Singapore", "Slovakia", "Slovenia", "South Africa", "Spain", "Sri Lanka", "Sweden", "Switzerland", "Thailand", "T\u00fcrkiye", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay"]

geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=countries_only, accuracy_per_country=False)

if not predict_coordinates:
    # And over all countries
    geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=None, accuracy_per_country=True)

/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Network-Model: efficientnet_b1
Project Name: region
Run ID: l964tv8w
Test Loss: 21.6985, Test Distance: 6594.8636, Test Distance Median: 6157.9355
Test Top 1 Accuracy: 0.0008, Test Top 3 Accuracy: 0.0037, Test Top 5 Accuracy: 0.0056
Test Top 1 Accuracy (Country): 0.0085, Test Top 3 Accuracy (Country): 0.0262, Test Top 5 Accuracy (Country): 0.0403
Test Top 1 Balanced Accuracy: 0.0010, Test Top 1 Balanced Accuracy (Country): 0.0107
Network-Model: efficientnet_b1
Project Name: region
Run ID: l964tv8w
Test Loss: 21.6985, Test Distance: 6594.8636, Test Distance Median: 6157.9355
Test Top 1 Accuracy: 0.0008, Test Top 3 Accuracy: 0.0037, Test Top 5 Accuracy: 0.0056
Test Top 1 Accuracy (Country): 0.0085, Test Top 3 Accuracy (Country): 0.0262, Test Top 5 Accuracy (Country): 0.0403
Test Top 1 Balanced Accuracy: 0.0010, Test Top 1 Balanced Accuracy (Country): 0.0107
Accuracy per country:
Country Romania: 0.05556
Country Peru: 0.03313
Country Bulgaria: 0.01777
Country Colombia: 0.01717
Country Por

/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/lukasstoeckli/GitLabProjects/dspro2/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
